### Importing Libraries

In [43]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


### Loading Datasets

In [44]:
training_data_int = pd.read_csv('data/training_data.csv', delimiter=';')
test_data_int = pd.read_csv('data/test_data_no_target.csv', delimiter=';')
group_dict = pd.read_csv('data/group_dictionary.csv', delimiter=';')
column_names_dict = pd.read_csv('data/column_names_dictionary.csv', delimiter=';')

### Data Preprocessing (Renaming column names, string-to-float)

In [46]:
# Edison's code for renaming column names
# Convert the column dictionary DataFrame to a dictionary
column_dict = dict(zip(column_names_dict['CODE'], column_names_dict['INDICATOR NAME']))
column_dict['Perform'] = 'Perform'  # Add mapping for 'perform' column
column_dict['Class'] = 'Class'      # Add mapping for 'class' column

# Map the encrypted column names to their original names using the column dictionary
training_data = training_data_int.rename(columns=column_dict)
test_data = test_data_int.rename(columns=column_dict)

numeric_columns = training_data.columns[1:-2] 

# Convert all columns except the first one to numeric
for column in numeric_columns:
    training_data[column] = training_data[column].str.replace(',', '.').astype(float)
    test_data[column] = test_data[column].str.replace(',', '.').astype(float)

training_data["Perform"] = training_data["Perform"].str.replace(',', '.').astype(float)

# Ensure columns are correctly targeted and all columns can be converted
try:
    numeric_columns = training_data.columns[1:-2]  # Adjust indices appropriately if needed

    # Convert all columns except the first and last two to float, handle non-string types
    for column in numeric_columns:
        training_data[column] = training_data[column].astype(str).str.replace(',', '.').astype(float)
        test_data[column] = test_data[column].astype(str).str.replace(',', '.').astype(float)

    # Convert 'Perform' column, ensure it's handled as a string if necessary
    if 'Perform' in training_data.columns:
        training_data['Perform'] = training_data['Perform'].astype(str).str.replace(',', '.').astype(float)

    print("Conversion successful.")

except Exception as e:
    print(f"An error occurred: {e}")
    
training_data.head()

Conversion successful.


,Industry sector,"Return on Average Total Assets - %, TTM","Return on Average Common Equity - %, TTM","EBITDA Percentage of Common Equity, TTM","EBITDA Percentage of Total Fixed Assets - Net, TTM",Excess Cash Margin - %,"Free Cash Flow Percentage of Total Revenue, TTM","Cash Flow from Operations Pct of Total Revenue, TTM","Cash Flow from Operations Pct of Common Equity, TTM","Cash Flow from Operations to Total Assets, TTM",...,1-year Absolute Change of Working Capital to Total Assets,1-year Absolute Change of Cash Ratio,1-year Absolute Change of Net Debt to Total Equity,1-year Absolute Change of Total Liabilities Percentage of Total Assets,1-year Absolute Change of Long Term Debt Percentage of Total Assets,1-year Absolute Change of Book Value Percentage of Market Capitalization,1-year Absolute Change of Cash Flow from Operations Pct of Capital Expenditures,"1-year Absolute Change of Price to Cash Flow from Operations per Share, TTM",Class,Perform
0,G9,0.136495,-0.028429,-0.037772,-0.232459,-0.016222,-0.187506,-0.322545,-0.043743,0.125389,...,0.102563,0.188481,-0.016027,-0.135451,-0.189667,0.250967,0.022171,-0.004265,-1,-0.033764
1,G5,-0.714522,-0.042137,-0.052968,-0.796862,-0.018394,0.070102,-0.076321,-0.063864,-1.045521,...,-0.016375,0.020727,-0.006525,-0.018790,-0.098543,0.317744,-0.180502,-0.009215,1,0.127771
2,G10,0.104791,-0.038188,-0.053191,0.620233,0.148587,0.489875,0.319274,-0.060246,0.053174,...,0.272937,0.774169,-0.007144,0.123954,0.000000,-0.110103,0.186669,-0.030720,1,0.183500
3,G2,-0.532847,-0.006582,-0.023377,1.306702,-0.068909,0.048024,-0.119481,-0.021057,-1.012916,...,0.004938,0.018494,-0.003350,-0.029214,0.045747,-0.076884,-0.037859,-0.012046,-1,-0.035668
4,G3,-0.200815,-0.016334,-0.036754,-0.886675,0.484495,-1.148744,0.152517,-0.043580,-0.935537,...,-0.014812,-0.324584,-0.019002,-0.379323,-0.046024,0.282145,0.011008,0.010496,1,0.235055


### Feature Engineering (one-hot encoding for 'Group' column)

In [49]:
# Check if 'Industry sector' column exists
if 'Industry sector' in training_data.columns:
    # Apply one-hot encoding
    industry_encoded = pd.get_dummies(training_data['Industry sector'], prefix='Industry sector')
    training_data = pd.concat([training_data, industry_encoded], axis=1)
    training_data.drop('Industry sector', axis=1, inplace=True)
    print("One-hot encoding successful for 'Industry sector' column.")
else:
    print("'Industry sector' column does not exist in the training dataset.")

if 'Industry sector' in test_data.columns:
    # Apply one-hot encoding
    industry_encoded = pd.get_dummies(test_data['Industry sector'], prefix='Industry sector')
    test_data = pd.concat([test_data, industry_encoded], axis=1)
    test_data.drop('Industry sector', axis=1, inplace=True)
    print("One-hot encoding successful for 'Industry sector' column.")
else:
    print("'Industry sector' column does not exist in the test dataset.")
    
training_data.head()


'Industry sector' column does not exist in the training dataset.
'Industry sector' column does not exist in the test dataset.


,"Return on Average Total Assets - %, TTM","Return on Average Common Equity - %, TTM","EBITDA Percentage of Common Equity, TTM","EBITDA Percentage of Total Fixed Assets - Net, TTM",Excess Cash Margin - %,"Free Cash Flow Percentage of Total Revenue, TTM","Cash Flow from Operations Pct of Total Revenue, TTM","Cash Flow from Operations Pct of Common Equity, TTM","Cash Flow from Operations to Total Assets, TTM","Reinvestment Rate - %, TTM",...,Industry sector_G10,Industry sector_G11,Industry sector_G2,Industry sector_G3,Industry sector_G4,Industry sector_G5,Industry sector_G6,Industry sector_G7,Industry sector_G8,Industry sector_G9
0,0.136495,-0.028429,-0.037772,-0.232459,-0.016222,-0.187506,-0.322545,-0.043743,0.125389,-0.014757,...,0,0,0,0,0,0,0,0,0,1
1,-0.714522,-0.042137,-0.052968,-0.796862,-0.018394,0.070102,-0.076321,-0.063864,-1.045521,-0.037353,...,0,0,0,0,0,1,0,0,0,0
2,0.104791,-0.038188,-0.053191,0.620233,0.148587,0.489875,0.319274,-0.060246,0.053174,-0.025008,...,1,0,0,0,0,0,0,0,0,0
3,-0.532847,-0.006582,-0.023377,1.306702,-0.068909,0.048024,-0.119481,-0.021057,-1.012916,-0.011783,...,0,0,1,0,0,0,0,0,0,0
4,-0.200815,-0.016334,-0.036754,-0.886675,0.484495,-1.148744,0.152517,-0.043580,-0.935537,-0.023262,...,0,0,0,1,0,0,0,0,0,0


### Handling Missing Values

In [50]:
# Setting up SimpleImputer to use the median strategy for numeric columns
imputer = SimpleImputer(strategy='median')

# Identify numeric columns that are common to both datasets
common_numeric_columns = training_data.select_dtypes(include=[np.number]).columns.intersection(test_data.select_dtypes(include=[np.number]).columns)

# Apply imputation to the training data
training_data[common_numeric_columns] = imputer.fit_transform(training_data[common_numeric_columns])

# Apply imputation to the test data (the same as in training dataset)
test_data[common_numeric_columns] = imputer.transform(test_data[common_numeric_columns])

print(training_data.isnull().sum().sum())
print(test_data.isnull().sum().sum())

0
0


### Edison's Code

In [ ]:
"""""
X = train_data["Perform"]
y = train_data["Class"] 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)

# Fit the model to the training data
clf.fit(pd.DataFrame(X_train), y_train)

# Make predictions on the testing data
y_pred = clf.predict(pd.DataFrame(X_test))

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# An accuracy of 1 is achieved, stacking method can be considered
"""

### Feature Engineering (Creating new var)

In [52]:
new_feature_names = ['debt_to_equity', 'current_ratio', 'return_on_assets', 'net_profit_margin']
if not all(col in training_data.columns for col in new_feature_names):
        new_features = pd.DataFrame({
            'debt_to_equity': training_data['EBITDA Percentage of Total Fixed Assets - Net, TTM'] / training_data['EBITDA Percentage of Common Equity, TTM'],
            'current_ratio': training_data['Excess Cash Margin - %'] / training_data['Free Cash Flow Percentage of Total Revenue, TTM'],
            'return_on_assets': training_data['Cash Flow from Operations Pct of Total Revenue, TTM'] / training_data['Cash Flow from Operations Pct of Common Equity, TTM'],
            'net_profit_margin': training_data['Cash Flow from Operations to Total Assets, TTM'] / training_data['Reinvestment Rate - %, TTM']
        })
        training_data = pd.concat([training_data, new_features], axis=1)
else:
    print("New features already exist in the DataFrame. Skipping creation in train data.")
training_data.head()

if not all(col in test_data.columns for col in new_feature_names):
        new_features = pd.DataFrame({
            'debt_to_equity': test_data['EBITDA Percentage of Total Fixed Assets - Net, TTM'] / test_data['EBITDA Percentage of Common Equity, TTM'],
            'current_ratio': test_data['Excess Cash Margin - %'] / test_data['Free Cash Flow Percentage of Total Revenue, TTM'],
            'return_on_assets': test_data['Cash Flow from Operations Pct of Total Revenue, TTM'] / test_data['Cash Flow from Operations Pct of Common Equity, TTM'],
            'net_profit_margin': test_data['Cash Flow from Operations to Total Assets, TTM'] / test_data['Reinvestment Rate - %, TTM']
        })
        test_data = pd.concat([test_data, new_features], axis=1)
else:
    print("New features already exist in the DataFrame. Skipping creation in test data.")
training_data.head()


New features already exist in the DataFrame. Skipping creation in train data.
New features already exist in the DataFrame. Skipping creation in test data.


,"Return on Average Total Assets - %, TTM","Return on Average Common Equity - %, TTM","EBITDA Percentage of Common Equity, TTM","EBITDA Percentage of Total Fixed Assets - Net, TTM",Excess Cash Margin - %,"Free Cash Flow Percentage of Total Revenue, TTM","Cash Flow from Operations Pct of Total Revenue, TTM","Cash Flow from Operations Pct of Common Equity, TTM","Cash Flow from Operations to Total Assets, TTM","Reinvestment Rate - %, TTM",...,Industry sector_G4,Industry sector_G5,Industry sector_G6,Industry sector_G7,Industry sector_G8,Industry sector_G9,debt_to_equity,current_ratio,return_on_assets,net_profit_margin
0,0.136495,-0.028429,-0.037772,-0.232459,-0.016222,-0.187506,-0.322545,-0.043743,0.125389,-0.014757,...,0.0,0.0,0.0,0.0,0.0,1.0,6.154294,0.086512,7.373653,-8.496662
1,-0.714522,-0.042137,-0.052968,-0.796862,-0.018394,0.070102,-0.076321,-0.063864,-1.045521,-0.037353,...,0.0,1.0,0.0,0.0,0.0,0.0,15.044252,-0.262381,1.195052,27.990212
2,0.104791,-0.038188,-0.053191,0.620233,0.148587,0.489875,0.319274,-0.060246,0.053174,-0.025008,...,0.0,0.0,0.0,0.0,0.0,0.0,-11.660528,0.303317,-5.299553,-2.126319
3,-0.532847,-0.006582,-0.023377,1.306702,-0.068909,0.048024,-0.119481,-0.021057,-1.012916,-0.011783,...,0.0,0.0,0.0,0.0,0.0,0.0,-55.895809,-1.434897,5.674097,85.964372
4,-0.200815,-0.016334,-0.036754,-0.886675,0.484495,-1.148744,0.152517,-0.043580,-0.935537,-0.023262,...,0.0,0.0,0.0,0.0,0.0,0.0,24.124850,-0.421760,-3.499725,40.217448


## Model Development

### Fixed Effects & Random Effects

In [115]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import confusion_matrix

training_data_effects_methods = training_data
test_data_effects_methods = test_data

# Clean up column names: replace spaces with underscores and prefix column names starting with a number
training_data_effects_methods.columns = ['_' + col if col[0].isdigit() else col for col in training_data_effects_methods.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]', '', regex=True)]
test_data_effects_methods.columns = ['_' + col if col[0].isdigit() else col for col in test_data_effects_methods.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]', '', regex=True)]

# Convert 'Class' to a categorical variable
training_data_effects_methods['Class'] = training_data_effects_methods['Class'].astype('category')

# Identify all numeric columns (excluding target columns 'Class' and 'Perform')
numeric_columns = training_data_effects_methods.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns.remove('Perform')

# Fixed Effects Model
# Create formula for fixed effects model
formula = 'Perform ~ ' + ' + '.join(numeric_columns)
fixed_effects_model = ols(formula, data=training_data_effects_methods).fit()

# Model Summary
fixed_effects_summary = fixed_effects_model.summary()

# Predicting 'Perform' using the fixed effects model
predictions = fixed_effects_model.predict(training_data_effects_methods[numeric_columns])

# Convert predictions to -1, 0, 1 based on thresholds
# Using median to split into three classes
median_perform = training_data_effects_methods['Perform'].median()
predictions_class = pd.cut(predictions, bins=[-float('inf'), -median_perform, median_perform, float('inf')], labels=[-1, 0, 1])

# Generate confusion matrix
true_classes = training_data_effects_methods['Class'].cat.codes  # Converting categories to numeric codes
pred_classes = predictions_class.cat.codes  # Converting categories to numeric codes
conf_matrix = confusion_matrix(true_classes, pred_classes, labels=[-1, 0, 1])

# Error cost matrix
cost_matrix = np.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]])

# Calculate error
error = np.sum(conf_matrix * cost_matrix) / len(true_classes)

fixed_effects_summary, error


(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                Perform   R-squared:                       0.039
 Model:                            OLS   Adj. R-squared:                  0.023
 Method:                 Least Squares   F-statistic:                     2.454
 Date:                Fri, 17 May 2024   Prob (F-statistic):           2.06e-17
 Time:                        21:47:45   Log-Likelihood:                 4164.0
 No. Observations:                8000   AIC:                            -8066.
 Df Residuals:                    7869   BIC:                            -7151.
 Df Model:                         130                                         
 Covariance Type:            nonrobust                                         
                                                                                          coef    std err          t      P>|t|      [0.025      0.975

In [94]:
# Assuming fixed_effects_model is already fitted

# Extract the summary table as a DataFrame
summary_df = fixed_effects_model.summary2().tables[1]

# Sort the summary table by the p-value in ascending order
sorted_summary_df = summary_df.sort_values(by='P>|t|')

# Display the first 20 rows
sorted_summary_df_head = sorted_summary_df.head(20)

sorted_summary_df_head


,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Cash_Flow_from_Operations_to_Total_Assets_TTM,0.018213,0.005491,3.316633,0.000915,0.007448,0.028977
Current_Asset_Turnover_TTM,0.010035,0.004057,2.473214,0.013411,0.002081,0.017988
Cash_Percentage_of_Total_Revenue_TTM,-0.006171,0.002499,-2.469697,0.013544,-0.011069,-0.001273
Gross_Profit_Margin___TTM,0.007434,0.003057,2.431668,0.015052,0.001441,0.013427
Net_Debt_to_EBITDA_TTM,-0.045362,0.019042,-2.382252,0.017231,-0.082689,-0.008035
Asset_Turnover_TTM,-0.013791,0.005843,-2.360443,0.018277,-0.025245,-0.002338
_1year_Absolute_Change_of_Free_Cash_Flow_Percentage_of_Total_Revenue_TTM,0.015901,0.007062,2.251572,0.024377,0.002057,0.029744
Net_Debt_to_Total_Equity,-0.016684,0.007636,-2.184884,0.028927,-0.031653,-0.001715
_1year_Absolute_Change_of_Net_Debt_to_Total_Equity,0.015036,0.007155,2.101498,0.035629,0.001011,0.029062
Inventories_Percentage_of_Total_Assets,0.007842,0.004154,1.887793,0.059090,-0.000301,0.015985


### Fixed Effects with Lasso Regression for feature selection

In [121]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(training_data_effects_methods[numeric_columns])
y = training_data_effects_methods['Perform']

# Lasso regression with cross-validation
lasso = LassoCV(cv=5, max_iter=50000, random_state=0).fit(X_scaled, y)

# Get the coefficients
lasso_coefficients = pd.Series(lasso.coef_, index=numeric_columns)

# Select features with non-zero coefficients
selected_features = lasso_coefficients[lasso_coefficients != 0].index.tolist()

# Apply the same transformations to the test dataset
#test_data_effects_methods[numeric_columns] = test_data_effects_methods[numeric_columns].apply(lambda x: x.str.replace(',', '.')).astype(float)
test_data_effects_methods_scaled = scaler.transform(test_data_effects_methods[numeric_columns])

# Create formula for the fixed effects model with selected features
formula = 'Perform ~ ' + ' + '.join(selected_features)
fixed_effects_model = ols(formula, data=training_data_effects_methods).fit()

# Model Summary
fixed_effects_summary = fixed_effects_model.summary()

# Predicting 'Perform' using the fixed effects model
test_data_effects_methods[selected_features] = test_data_effects_methods_scaled[:, [numeric_columns.index(feat) for feat in selected_features]]
test_predictions = fixed_effects_model.predict(test_data_effects_methods[selected_features])


# Convert predictions to -1, 0, 1 based on thresholds
median_perform = training_data_effects_methods['Perform'].median()
test_predictions_class = pd.cut(test_predictions, bins=[-float('inf'), -median_perform, median_perform, float('inf')], labels=[-1, 0, 1])

# Save predictions to a CSV file in the required format
#submission = pd.DataFrame(test_predictions_class, columns=['Predicted'])
#submission.to_csv('data/submission_fixed_effects.csv', index=False, header=False)




### Random Effects Model

In [79]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from statsmodels.regression.mixed_linear_model import MixedLM
from statsmodels.tools import add_constant

# Load the training and test datasets
training_data_random_effects = training_data
test_data_random_effects = test_data


# Perform one-hot encoding for 'Group'
industry_encoded_train = pd.get_dummies(training_data_int['Group'], prefix='Industry_sector')
industry_encoded_test = pd.get_dummies(test_data_int['Group'], prefix='Industry_sector')

# Check if the one-hot encoded columns exist in the preprocessed dataset before dropping them
encoded_columns_train = list(industry_encoded_train.columns)
encoded_columns_test = list(industry_encoded_test.columns)

if all(col in training_data_random_effects.columns for col in encoded_columns_train):
    training_data_random_effects.drop(encoded_columns_train, axis=1, inplace=True)
    
if all(col in test_data_random_effects.columns for col in encoded_columns_test):
    test_data_random_effects.drop(encoded_columns_test, axis=1, inplace=True)

# Add back the original group column to the preprocessed dataset if it does not exist
if 'Group' not in training_data_random_effects.columns:
    training_data_random_effects['Group'] = training_data_int['Group']

if 'Group' not in test_data_random_effects.columns:
    test_data_random_effects['Group'] = test_data_int['Group']

# Convert necessary columns to numeric
numeric_columns = training_data_random_effects.select_dtypes(include=[np.number]).columns.tolist()

for col in ['Perform', 'Class']:
    if col in numeric_columns:
        numeric_columns.remove(col)

# Standardize the features and the target variable
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(training_data_random_effects[numeric_columns])
y_scaled = scaler_y.fit_transform(training_data_random_effects[['Perform']]).flatten()

# Lasso regression with cross-validation
lasso = LassoCV(cv=5, max_iter=50000, random_state=0).fit(X_scaled, y)

# Get the coefficients
lasso_coefficients = pd.Series(lasso.coef_, index=numeric_columns)

# Select features with non-zero coefficients
selected_features = lasso_coefficients[lasso_coefficients != 0].index.tolist()

# Create the design matrix for the random effects model
X = add_constant(training_data_random_effects[selected_features])
groups = training_data_random_effects['Group']

# Fit the random effects model
random_effects_model = MixedLM(endog=training_data_random_effects['Perform'], exog=X, groups=groups).fit()

# Model Summary
random_effects_summary = random_effects_model.summary()

# Apply the same standardization to the test dataset
test_data_random_effects_scaled = scaler_X.transform(test_data_random_effects[numeric_columns])
test_data_random_effects[selected_features] = test_data_random_effects_scaled[:, [numeric_columns.index(feat) for feat in selected_features]]


# Create the design matrix for the test dataset
X_test = add_constant(test_data_random_effects[selected_features])

# Predicting 'Perform' using the random effects model
test_predictions_scaled = random_effects_model.predict(X_test)

# Convert test_predictions_scaled to a NumPy array and reshape it
test_predictions_scaled_array = test_predictions_scaled.values.reshape(-1, 1)

# Inverse transform the scaled predictions to original scale
test_predictions = scaler_y.inverse_transform(test_predictions_scaled_array).flatten()

# Convert predictions to -1, 0, 1 based on thresholds
median_perform = training_data_random_effects['Perform'].median()
test_predictions_class = pd.cut(test_predictions, bins=[-float('inf'), -median_perform, median_perform, float('inf')], labels=[-1, 0, 1])

# Save predictions to a CSV file in the required format
submission = pd.DataFrame(test_predictions_class, columns=['Predicted'])
submission.to_csv('data/submission_random_effects.csv', index=False, header=False)


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
